In [1]:
from bs4 import BeautifulSoup, NavigableString
import requests
from time import sleep
from random import randint
import math
from tqdm import tqdm
import pickle
import pandas as pd
from datetime import date
from datetime import datetime
import json

In [9]:
# initialize df 

columns = ['id','title', 'company', 'time', 'job_description', 'seniority_level', 'job_function', 'industries', 'Scraping_date', 'url', 'keyword']

scraper_df = pd.DataFrame(columns=columns)
print(scraper_df)

current_date = datetime.now()
time = current_date.strftime("%Y-%m-%d")
time

todays_scrap = {}
scraper_df_json = scraper_df.to_json('dataframe.json', orient='split', date_format='iso', indent=4)

print(scraper_df_json)
todays_scrap[time] = scraper_df_json

todays_scrap

# with open(f'webscraper/webscrap_data/webscrap.json', 'wb') as file:
#     json.dump(todays_scrap, file=file)


Empty DataFrame
Columns: [id, title, company, time, job_description, seniority_level, job_function, industries, Scraping_date, url, keyword]
Index: []
None


{'2023-09-20': None}